In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

def count_original_images(source_dir):
    class_counts = {}
    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)
        if not os.path.isdir(class_path) or class_name == 'split_csv':
            continue
        
        class_counts[class_name] = {'normal': 0, 'anomaly': 0}
        
        normal_path = os.path.join(class_path, "Data", "Images", "Normal")
        anomaly_path = os.path.join(class_path, "Data", "Images", "Anomaly")
        
        if os.path.exists(normal_path):
            class_counts[class_name]['normal'] = len([f for f in os.listdir(normal_path) if not f.endswith('.csv')])
        if os.path.exists(anomaly_path):
            class_counts[class_name]['anomaly'] = len([f for f in os.listdir(anomaly_path) if not f.endswith('.csv')])
    
    return class_counts

def restructure_dataset(source_dir, dest_dir, test_size=0.2):
    os.makedirs(dest_dir, exist_ok=True)
    class_counts = {}

    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)
        if not os.path.isdir(class_path) or class_name == 'split_csv':
            continue

        new_class_dir = os.path.join(dest_dir, f"class_{class_name}")
        os.makedirs(new_class_dir, exist_ok=True)

        train_dir = os.path.join(new_class_dir, "train")
        test_dir = os.path.join(new_class_dir, "test")
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)

        os.makedirs(os.path.join(train_dir, "good"), exist_ok=True)
        os.makedirs(os.path.join(test_dir, "good"), exist_ok=True)
        os.makedirs(os.path.join(test_dir, "anomaly"), exist_ok=True)

        class_counts[class_name] = {'train_good': 0, 'test_good': 0, 'test_anomaly': 0}

        try:
            normal_images = [f for f in os.listdir(os.path.join(class_path, "Data", "Images", "Normal")) 
                             if not f.endswith('.csv')]
        except FileNotFoundError:
            print(f"Warning: Normal images directory not found for {class_name}. Skipping.")
            continue

        train_images, test_images = train_test_split(normal_images, test_size=test_size, random_state=42)

        for img in train_images:
            src = os.path.join(class_path, "Data", "Images", "Normal", img)
            dst = os.path.join(train_dir, "good", img)
            shutil.copy(src, dst)
            class_counts[class_name]['train_good'] += 1

        for img in test_images:
            src = os.path.join(class_path, "Data", "Images", "Normal", img)
            dst = os.path.join(test_dir, "good", img)
            shutil.copy(src, dst)
            class_counts[class_name]['test_good'] += 1

        try:
            anomaly_images = [f for f in os.listdir(os.path.join(class_path, "Data", "Images", "Anomaly")) 
                              if not f.endswith('.csv')]
        except FileNotFoundError:
            print(f"Warning: Anomaly images directory not found for {class_name}. Skipping.")
            continue

        for img in anomaly_images:
            src = os.path.join(class_path, "Data", "Images", "Anomaly", img)
            dst = os.path.join(test_dir, "anomaly", img)
            shutil.copy(src, dst)
            class_counts[class_name]['test_anomaly'] += 1

    print("Dataset restructuring completed.")
    return class_counts

def count_restructured_images(directory):
    class_counts = {}
    for class_dir in os.listdir(directory):
        if not class_dir.startswith('class_'):
            continue
        class_name = class_dir[6:]  # Remove 'class_' prefix
        class_counts[class_name] = {'train_good': 0, 'test_good': 0, 'test_anomaly': 0}
        class_path = os.path.join(directory, class_dir)
        
        train_good_path = os.path.join(class_path, 'train', 'good')
        test_good_path = os.path.join(class_path, 'test', 'good')
        test_anomaly_path = os.path.join(class_path, 'test', 'anomaly')
        
        class_counts[class_name]['train_good'] = len([f for f in os.listdir(train_good_path) if not f.endswith('.csv')])
        class_counts[class_name]['test_good'] = len([f for f in os.listdir(test_good_path) if not f.endswith('.csv')])
        class_counts[class_name]['test_anomaly'] = len([f for f in os.listdir(test_anomaly_path) if not f.endswith('.csv')])
    
    return class_counts

def verify_counts(original, restructured):
    print("\nImage Count Verification:")
    print("--------------------------")
    all_match = True

    for class_name in original.keys():
        print(f"\nClass: {class_name}")
        print("  Category    | Original | Restructured")
        print("-------------|----------|-------------")
        
        orig_normal = original[class_name]['normal']
        orig_anomaly = original[class_name]['anomaly']
        
        restr_train_good = restructured[class_name]['train_good']
        restr_test_good = restructured[class_name]['test_good']
        restr_test_anomaly = restructured[class_name]['test_anomaly']
        
        print(f"  Normal      | {orig_normal:8d} | {restr_train_good + restr_test_good:11d}")
        print(f"  Anomaly     | {orig_anomaly:8d} | {restr_test_anomaly:11d}")
        
        if orig_normal != restr_train_good + restr_test_good or orig_anomaly != restr_test_anomaly:
            all_match = False
            print("  Mismatch detected!")
        
        print("-------------|----------|-------------")
        print(f"  Total       | {orig_normal + orig_anomaly:8d} | {restr_train_good + restr_test_good + restr_test_anomaly:11d}")

    if all_match:
        print("\nAll image counts match between original and restructured datasets!")
    else:
        print("\nDiscrepancy detected in image counts between original and restructured datasets.")


source_directory = "/teamspace/studios/this_studio/VisA"
destination_directory = "/teamspace/studios/this_studio/Dataset"

# Count images in the original dataset
original_counts = count_original_images(source_directory)

# Restructure the dataset
restructure_dataset(source_directory, destination_directory)

# Count images in the restructured dataset
restructured_counts = count_restructured_images(destination_directory)

# Verify the counts
verify_counts(original_counts, restructured_counts)

Dataset restructuring completed.

Image Count Verification:
--------------------------

Class: macaroni1
  Category    | Original | Restructured
-------------|----------|-------------
  Normal      |     1000 |        1000
  Anomaly     |      100 |         100
-------------|----------|-------------
  Total       |     1100 |        1100

Class: fryum
  Category    | Original | Restructured
-------------|----------|-------------
  Normal      |      500 |         500
  Anomaly     |      100 |         100
-------------|----------|-------------
  Total       |      600 |         600

Class: candle
  Category    | Original | Restructured
-------------|----------|-------------
  Normal      |     1000 |        1000
  Anomaly     |      100 |         100
-------------|----------|-------------
  Total       |     1100 |        1100

Class: pcb1
  Category    | Original | Restructured
-------------|----------|-------------
  Normal      |     1004 |        1004
  Anomaly     |      100 |    

In [15]:
!git clone https://github.com/marco-rudolph/differnet.git

Cloning into 'differnet'...



# Train Classes

In [3]:
!python main.py

Starting training on class: class_candle
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Train epoch 0
100%|███████████████████████████████████████████| 11/11 [00:39<00:00,  3.62s/it]
Epoch: 0.0 	 train loss: 0.1154
100%|███████████████████████████████████████████| 11/11 [00:28<00:00,  2.56s/it]
Epoch: 0.1 	 train loss: -1.3861
100%|█████████████████████████████████████

## From chewinggum

In [2]:
!python main.py

Starting training on class: class_chewinggum
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Train epoch 0
100%|█████████████████████████████████████████████| 6/6 [00:22<00:00,  3.82s/it]
Epoch: 0.0 	 train loss: 0.3556
100%|█████████████████████████████████████████████| 6/6 [00:18<00:00,  3.15s/it]
Epoch: 0.1 	 train loss: -0.4254
100%|█████████████████████████████████

# Pipe fryum Class

In [3]:
!python main.py

Starting training on class: class_pipe_fryum
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /home/zeus/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|█████████████████████████████████████████| 233M/233M [00:01<00:00, 194MB/s]

Train epoch 0
100%|███████████████████████████████████

## pcb4 Class

In [2]:
!python main.py

Starting training on class: class_pcb4
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Train epoch 0
100%|███████████████████████████████████████████| 11/11 [00:40<00:00,  3.65s/it]
Epoch: 0.0 	 train loss: 0.0609
100%|███████████████████████████████████████████| 11/11 [00:30<00:00,  2.76s/it]
Epoch: 0.1 	 train loss: -0.9721
100%|███████████████████████████████████████

## pcb3 Class

In [3]:
!python main.py

Starting training on class: class_pcb3
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Train epoch 0
100%|███████████████████████████████████████████| 11/11 [00:36<00:00,  3.29s/it]
Epoch: 0.0 	 train loss: 0.1564
100%|███████████████████████████████████████████| 11/11 [00:26<00:00,  2.42s/it]
Epoch: 0.1 	 train loss: -0.7842
100%|███████████████████████████████████████

## Macaroni 1 and Macaroni 2 , PCB1

In [ ]:
!python main.py

Starting training on class: class_macaroni1
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Train epoch 0
100%|███████████████████████████████████████████| 10/10 [00:38<00:00,  3.88s/it]
Epoch: 0.0 	 train loss: 0.1421
100%|███████████████████████████████████████████| 10/10 [00:25<00:00,  2.57s/it]
Epoch: 0.1 	 train loss: -0.9113
100%|██████████████████████████████████

# PCB2

In [ ]:
!python main.py

Starting training on class: class_pcb2
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Train epoch 0
100%|███████████████████████████████████████████| 10/10 [00:37<00:00,  3.72s/it]
Epoch: 0.0 	 train loss: 0.1602
100%|███████████████████████████████████████████| 10/10 [00:28<00:00,  2.83s/it]
Epoch: 0.1 	 train loss: -0.8506
100%|███████████████████████████████████████